Mahyar Bayran (260833355) mahyar.bayran@mail.mcgill.ca  
Peter Quinn (260689207) peter.quinn@mail.mcgill.ca  
COMP 767 Project  
W2020 

# **Tic-Tac-Toe**:

In [0]:
# imports
import numpy as np
from matplotlib import pyplot as plt
from math import floor
from math import sqrt
%matplotlib inline
plt.rcParams["figure.figsize"] = (16.1,10)
from math import pow, log, sqrt

In [3]:
import csv
from google.colab import drive

drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


# **Functions needed:**

In [0]:
# mapping between state and range(0, 3**9)
def stateToDecimal(ss):
  id = 0
  x = 1
  for i in range(0,3):
    y = 1
    for j in range(0,3):
      id += ss[i][j] * x * y
      y *= 3
    x *= 27
  return int(id)

# check to see if we have a winner
def checkWinner(ss):
  # rows
  result = (ss[0][0] == ss[0][1] and ss[0][0] == ss[0][2] and (not ss[0][0]==0))
  result = result or (ss[1][0] == ss[1][1] and ss[1][0] == ss[1][2] and (not ss[1][0]==0))
  result = result or (ss[2][0] == ss[2][1] and ss[2][0] == ss[2][2] and (not ss[2][0]==0))
  # columns
  result = result or (ss[0][0] == ss[1][0] and ss[0][0] == ss[2][0] and (not ss[0][0]==0))
  result = result or (ss[0][1] == ss[1][1] and ss[0][1] == ss[2][1] and (not ss[0][1]==0))
  result = result or (ss[0][2] == ss[1][2] and ss[0][2] == ss[2][2] and (not ss[0][2]==0))
  # diagonal
  result = result or (ss[0][0] == ss[1][1] and ss[0][0] == ss[2][2] and (not ss[0][0]==0))
  result = result or (ss[0][2] == ss[1][1] and ss[0][2] == ss[2][0] and (not ss[0][2]==0))
  return result

## based on the state, some actions are not available
def getAvailableActions(ss):
  AA = []
  for i in range(0,3):
    for j in range(0,3):
      if ss[i][j] == 0:
        AA.append(3*i + j)
  return AA

## swap 1s with 2s, 2s with 1s
def invertState(ss):
  return np.remainder(np.add(-ss, 3), 3)

## choose an action according to a soft-policy
## UCB
def policyUCB(ss, t):
  c = 1
  A = getAvailableActions(ss)
  L = len(A)
  if L == 0:
    return -1
  QQ = np.zeros(L)
  for i in range(0, L):
    n = N[stateToDecimal(ss), A[i]]
    if n == 0:
      QQ[i] = 100000 # give a high priority to a never selected state-action pair
    else:
      QQ[i] = Q[stateToDecimal(ss), A[i]] + c * sqrt(log(t)/n)

  best_a_i = np.argmax(QQ)
  worst_a_i = np.argmin(QQ)
  if QQ[best_a_i] == QQ[worst_a_i]:
    # choose  uniformly
    return A[np.random.randint(L)]
  else:
    return A[best_a_i]

def epsGreedy(ss):
  A = getAvailableActions(ss)
  L = len(A)
  if L == 0:
    return -1, 0.0
  QQ = np.zeros(L)
  for i in range(0, L):
      QQ[i] = Q[stateToDecimal(ss), A[i]]

  best_a_i = np.argmax(QQ)
  worst_a_i = np.argmin(QQ)
  if QQ[best_a_i] == QQ[worst_a_i]:
    # choose  uniformly
    return A[np.random.randint(L)], 1/L
  else:
    # choose according to eps-greedy
    P = np.zeros(L)
    P = np.add(P, EPS/L)
    P[best_a_i] += 1.0 - EPS
    picked_action_i = np.random.choice(list(range(0,L)), p=list(P))
    return A[picked_action_i], P[picked_action_i]

def greedy(ss):
  A = getAvailableActions(ss)
  #print('ss: ', ss)
  #print('available actions :', A)
  L = len(A)
  if L == 0:
    return -1
  QQ = np.zeros(L)
  for i in range(0, L):
    QQ[i] = Q[stateToDecimal(ss), A[i]]

  best_a_i = np.argmax(QQ)
  worst_a_i = np.argmin(QQ)
  if QQ[best_a_i] == QQ[worst_a_i]:
    # choose  uniformly
    return A[np.random.randint(L)]
  else:
    return A[best_a_i]

# **Parameters**:

In [0]:
# discount factor
gamma = 0.9

# learning rate
alpha = 1

# eps-greedy
EPS = 0.1

# number of episodes to run for
E = int(20e6)

# **Training:**

**SARSA:**

In [0]:
# Initialize Q-Values
# Q function assumes that we always have the role = 0.
# Therefore when role = 1, we find the Q value for the inverted state
Q = {}
N = {}
for i in range(0, 3**9):
  for act in range(0, 9):
    Q[i,act] = 0.0
    N[i,act] = 0.0

# ensure reproducibility
np.random.seed(10)

# number of wins
wins = [0, 0]
ties = 0

# at each turn the agent will have a roll
# 0: o, 1: X
# initialize the role
role = 0; # at the end of each turn role = 1 - role

for e in range(0,E):
  # initialize states
  s = np.zeros((3,3))
  terminated = False
  t = 1

  if e%10000 == 0:
    print('episode: ', e)

  # s holds the true state
  # X holds the state depending on the role (inverse or not)
  
  # take the first action
  # choose action wrt the policy and role
  X1 = np.copy(s)
  if role == 1:   
    X1 = invertState(X1)
  
  a1 = policyUCB(X1, t)
  x1 = stateToDecimal(X1)

  # debugging UCB
  #if e < 12:
  #  print("a1 : ", a1)

  N[x1, a1] += 1

  # loop in each episode
  while(not terminated):
    
    # get next state
    i, j = divmod(a1, 3)
    s_next = np.copy(s)
    s_next[i][j] = role + 1
    t += 1

    X1 = np.copy(s)
    if role == 1:   
      X1 = invertState(X1)
    x1 = stateToDecimal(X1)
    
    if e > E :
      print('whose turn?', role + 1)

      print('current state: \n', s)
      #print('state after action: \n', s_next)

    # get reward
    r = 0.0
    # check termination
    terminated = checkWinner(s_next)
    if terminated:
      r = 100.0
      wins[role] += 1
      #print('wins: ', wins)

    # update Q values
    Q1 = Q[x1, a1]
    # whatever the next player does is gonna harm the current player
    # so if s_next, a2 has high Q value, then a,s should not be taken!
    # so we should negate Q2
    # reminder: when checking Q values, input state should be wrt role=0
    X2 = np.copy(s_next)
    if role == 0:
      X2 = invertState(X2)
    
    # choose an action
    a2 = policyUCB(X2, t)
    
    if a2 == -1:
      Q2 = 0.0
      if not terminated:
        # it's a tie
        ties += 1
        #print('tied', ties)
        # swap roles so that the other one can start
        role = 1 - role
        terminated = True 
    else:
      x2 = stateToDecimal(X2)
      Q2 = -Q[x2, a2]
      N[x2, a2] += 1

    if terminated:
      Q2 = 0

    Q[x1, a1] = Q1 + alpha * (r + gamma * Q2 - Q1)

    #print('X1: \n', X1)
    #print('X2: \n', X2)
    #print('x1: ', x1, ',a1: ', a1, ',x2: ', x2, ', a2: ', a2)
    #print('Q1: ', Q1 , ',Q2: ', Q2, ',r: ', r, ', new Q: ', Q[x1, a1])

    #print('New Q value for current action and state: ',Q[x1,a1])

    # update current variables
    s = s_next
    a1 = a2

    # swap role (X to O, O to X) if not terminated
    # if terminated don't switch roles so that the winner can begin
    if not terminated:
      role = 1 - role
    
    #print('terminated? ', terminated)
    #print('')


print('wins', wins)
print('ties', ties)
print('\n\n')


# SARSA: Save Q-Values to File ##
with (open('/gdrive/My Drive/Project767/SARSA_Q_VALUES.csv', 'w')) as f:
  w = csv.writer(f)
  for key, val in Q.items():
    w.writerow([key[0], key[1], val])

episode:  0
episode:  10000
episode:  20000
episode:  30000
episode:  40000
episode:  50000
episode:  60000
episode:  70000
episode:  80000
episode:  90000
episode:  100000
episode:  110000
episode:  120000
episode:  130000
episode:  140000
episode:  150000
episode:  160000
episode:  170000
episode:  180000
episode:  190000
episode:  200000
episode:  210000
episode:  220000
episode:  230000
episode:  240000
episode:  250000
episode:  260000
episode:  270000
episode:  280000
episode:  290000
episode:  300000
episode:  310000
episode:  320000
episode:  330000
episode:  340000
episode:  350000
episode:  360000
episode:  370000
episode:  380000
episode:  390000
episode:  400000
episode:  410000
episode:  420000
episode:  430000
episode:  440000
episode:  450000
episode:  460000
episode:  470000
episode:  480000
episode:  490000
episode:  500000
episode:  510000
episode:  520000
episode:  530000
episode:  540000
episode:  550000
episode:  560000
episode:  570000
episode:  580000
episode:  5

# **Off-Policy Monte Carlo**

In [6]:
# Initialize Q-Values
# Q function assumes that we always have the role = 0.
# Therefore when role = 1, we find the Q value for the inverted state
Q = {}
# number of times action taken
N = {}
C = {}
for i in range(0, 3**9):
  for act in range(0, 9):
    Q[i,act] = 0.0
    N[i,act] = 0.0
    C[i,act] = 0.0

# ensure reproducibility
np.random.seed(100)

def generateEpisode(ep_states, ep_xs, ep_actions, ep_rewards, ep_bs):
  # initialize the role
  role = 0

  global wins
  global ties

  # initialize states
  s = np.zeros((3,3))
  terminated = False
  t = 1

  # s holds the true state
  # X holds the state depending on the role (inverse or not)

  # loop in each episode
  while(not terminated):

    # choose action wrt the policy and role
    X1 = np.copy(s)
    if role == 1:   
      X1 = invertState(X1)

    a1, b = epsGreedy(X1)
    
    x1 = stateToDecimal(X1)
      
    #print('s : ', s)
    #print('a : ' ,a1)
    t += 1

    # get reward
    r = 0.0
    

    if a1 == -1:
      ties += 1
      role = 1 - role
      terminated = True 
    else:
      N[x1, a1] += 1
      # get next state
      i, j = divmod(a1, 3)
      s_next = np.copy(s)
      s_next[i][j] = role + 1
      # check termination
      terminated = checkWinner(s_next)
      if terminated:
        r = 100.0
        wins[role] += 1
      
      ep_states.append(X1)
      ep_xs.append(x1)
      ep_actions.append(a1)
      ep_rewards.append(r)
      ep_bs.append(b)
    

    # update current variables
    s = s_next

    # swap role (X to O, O to X) if not terminated
    # if terminated don't switch roles so that the winner can begin
    if not terminated:
      role = 1 - role
    

# number of wins
wins = [0, 0]
ties = 0

for e in range(0,E):

  if e%10000 == 0:
    print('episode: ', e)
  
  ep_states = []
  ep_xs = []
  ep_actions = []
  ep_rewards = []
  ep_bs = []

  # generate episode using soft policy
  generateEpisode(ep_states, ep_xs, ep_actions, ep_rewards, ep_bs)

  #print(ep_states)
  #print(ep_xs)
  #print(ep_actions)
  #print(ep_rewards)
  #print(ep_bs)

  # initialize return 
  G = 0

  W = 1

  T = len(ep_xs)

  sgnR = 1

  for t in reversed(range(0,T)):
    X = ep_states[t]
    x = ep_xs[t]
    a = ep_actions[t]
    r = ep_rewards[t]
    b = ep_bs[t]

    G = - gamma * G + r
    
    C[x,a] += W
    Q[x,a] += W/C[x,a] * (G - Q[x,a])
    #print('X: ')
    

    pi_s = greedy(X)
    if pi_s != a:
      break

    W = W / b

print('wins', wins)
print('ties', ties)
print('\n\n')

## Off-Policy Monte Carlo: Save Q-Values to File ##
with (open('/gdrive/My Drive/Project767/OffMC_Q_VALUES.csv', 'w')) as f:
  w = csv.writer(f)
  for key, val in Q.items():
    w.writerow([key[0], key[1], val])

episode:  0
episode:  10000
episode:  20000
episode:  30000
episode:  40000
episode:  50000
episode:  60000
episode:  70000
episode:  80000
episode:  90000
episode:  100000
episode:  110000
episode:  120000
episode:  130000
episode:  140000
episode:  150000
episode:  160000
episode:  170000
episode:  180000
episode:  190000
episode:  200000
episode:  210000
episode:  220000
episode:  230000
episode:  240000
episode:  250000
episode:  260000
episode:  270000
episode:  280000
episode:  290000
episode:  300000
episode:  310000
episode:  320000
episode:  330000
episode:  340000
episode:  350000
episode:  360000
episode:  370000
episode:  380000
episode:  390000
episode:  400000
episode:  410000
episode:  420000
episode:  430000
episode:  440000
episode:  450000
episode:  460000
episode:  470000
episode:  480000
episode:  490000
episode:  500000
episode:  510000
episode:  520000
episode:  530000
episode:  540000
episode:  550000
episode:  560000
episode:  570000
episode:  580000
episode:  5

# **Test with greedy actions and learned Q-values**

In [0]:
# number of wins
wins = [0, 0]
ties = 0

# at each turn the agent will have a roll
# 0: o, 1: X
# initialize the role
role = 0; # at the end of each turn role = 1 - role

for e in range(0,1):
  # initialize states
  s = np.zeros((3,3))
  terminated = False

  X1 = np.copy(s)
  if role == 1:   
    X1 = invertState(X1)
  
  a1 = greedy(X1)
  x1 = stateToDecimal(X1)

  print('episode :', e)
  print('action: ', a1)
  print('Q value for x1 and a1: ', Q[x1,a1])

  # loop in each episode
  while(not terminated):
    
    # get next state
    i, j = divmod(a1, 3)
    s_next = np.copy(s)
    s_next[i][j] = role + 1

    if e > -1:
      print('whose turn?', role + 1)

      #print('current state: \n', s)
      print('action: ', a1)
      print('state after action:')
      print(s_next)

    
    # check termination
    terminated = checkWinner(s_next)
    if terminated:
      wins[role] += 1
      #print('wins: ', wins)

    X2 = np.copy(s_next)
    if role == 1:
      X2 = invertState(X2)
    
    # choose an action
    a2 = greedy(X2)

    #print('action: ', a2)

    if a2 == -1:
      if not terminated:
        # it's a tie
        ties += 1
        #print('tied', ties)
        # swap roles so that the other one can start
        role = 1 - role
        terminated = True 
    else:
      x2 = stateToDecimal(X2)
      print('a2: ', a2)
      print('Q value for x2 and a2: ', Q[x2, a2])
      
    print('')
    # update current variables
    s = s_next
    a1 = a2
    x1 = x2
    
    # swap role (X to O, O to X) if not terminated
    # if terminated don't switch roles so that the winner can begin
    if not terminated:
      role = 1 - role


print('wins', wins)
print('ties', ties)
print('\n\n')